In [1]:
# Dependencies
import pandas as pd
import requests as req
import os, json, csv, time

In [2]:
# Assign the file to a variable 
file_path = os.path.join("static", "data", "Canada_transforming.csv")

# Read the file
open_file = pd.read_csv(file_path, encoding="latin1") 

# Assign file to a DataFrame
df = pd.DataFrame(open_file)

# Preview the DataFrame
df.head()

,id_area,areaname,id_reg,regname,country,id_dev,devname,1980,1981,1982,...,2012,2013,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,935.0,Asia,5501.0,Southern Asia,Afghanistan,902.0,Developing regions,16.0,39.0,39.0,...,2635.0,2004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,908.0,Europe,925.0,Southern Europe,Albania,901.0,Developed regions,1.0,0.0,0.0,...,620.0,603.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,903.0,Africa,912.0,Northern Africa,Algeria,902.0,Developing regions,80.0,67.0,71.0,...,3774.0,4331.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,909.0,Oceania,957.0,Polynesia,American Samoa,902.0,Developing regions,0.0,1.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,908.0,Europe,925.0,Southern Europe,Andorra,901.0,Developed regions,0.0,0.0,0.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Treatment

In [3]:
# Get just the columns we will use for the project
df = df[['areaname', 'regname', 'country', 'devname', '1980', '1981', '1982', '1983', '1984', '1985', '1986', \
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',\
        '2000', '2001', '2002', '2003', '2004','2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']]

# Remove blanck rows in the end of the DataFrame
df.dropna(subset=["country"], inplace=True)

df.tail()

,areaname,regname,country,devname,1980,1981,1982,1983,1984,1985,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
189,Asia,South-Eastern Asia,Viet Nam,Developing regions,1191.0,1829.0,2162.0,3404.0,7583.0,5907.0,...,1816.0,1852.0,3153.0,2574.0,1784.0,2171.0,1942.0,1723.0,1731.0,2112.0
190,Africa,Northern Africa,Western Sahara,Developing regions,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191,Asia,Western Asia,Yemen,Developing regions,1.0,2.0,1.0,6.0,0.0,18.0,...,124.0,161.0,140.0,122.0,133.0,128.0,211.0,160.0,174.0,217.0
192,Africa,Eastern Africa,Zambia,Developing regions,11.0,17.0,11.0,7.0,16.0,9.0,...,56.0,91.0,77.0,71.0,64.0,60.0,102.0,69.0,46.0,59.0
193,Africa,Eastern Africa,Zimbabwe,Developing regions,72.0,114.0,102.0,44.0,32.0,29.0,...,1450.0,615.0,454.0,663.0,611.0,508.0,494.0,434.0,437.0,407.0


### Create new fields and Request the API to fill them

In [4]:
# Create the new columns
df["lat"] = 0.0
df["lon"] = 0.0
df["flag"] = ""
df["capital"] = ""
df["population"] = ""
df["countrycode"] = ""


In [5]:
# Set the API URL to get Latitude, Longitude and the SVG country map
url = f"https://restcountries.eu/rest/v2/name/"

# Fill new fields. 
print("-----------------------------")
print("Beginning Data Retrieval")
print("-----------------------------")       

# Loop to execute requests for each country. 
count = 0 
for index, row in df.iterrows(): 
    # Uncomment the three lines below to process just some rows. 
#     if count == 5:
#         break
#     count += 1
        
    # Add one second interval between requests to avoid overload the API 
    time.sleep(1) 
    
    # Mount the query URL for current country
    query_url = f"{url}{row['country']}?fullText=true"
    
    # print the counter and current country. 
    country = f"{row['country']}" 
    print(f"Processing Record {count} - ({row['country']})")
    # print the URL
    print(query_url)

    try:
        # Run the request
        result = req.get(query_url).json()
        
        # Print a column to check the result
        print(result[0]['flag'])
        
        # Fill fields
        df.loc[index, 'lat']  = result[0]['latlng'][0]
        df.loc[index, 'lon']  = result[0]['latlng'][1]
        df.loc[index, 'flag'] = result[0]['flag']
        df.loc[index, 'capital'] = result[0]['capital']
        df.loc[index, 'population'] = result[0]['population']
        df.loc[index, 'countrycode'] = result[0]['alpha2Code']
        
    except:    
        print(f"We've got an error when processing country={country} - {result}.")
        # Remove the country with error from DF.
        df.drop([index],inplace=True)


-----------------------------
Beginning Data Retrieval
-----------------------------
Processing Record 0 - (Afghanistan)
https://restcountries.eu/rest/v2/name/Afghanistan?fullText=true
https://restcountries.eu/data/afg.svg
Processing Record 0 - (Albania)
https://restcountries.eu/rest/v2/name/Albania?fullText=true
https://restcountries.eu/data/alb.svg
Processing Record 0 - (Algeria)
https://restcountries.eu/rest/v2/name/Algeria?fullText=true
https://restcountries.eu/data/dza.svg
Processing Record 0 - (American Samoa)
https://restcountries.eu/rest/v2/name/American Samoa?fullText=true
https://restcountries.eu/data/asm.svg
Processing Record 0 - (Andorra)
https://restcountries.eu/rest/v2/name/Andorra?fullText=true
https://restcountries.eu/data/and.svg
Processing Record 0 - (Angola)
https://restcountries.eu/rest/v2/name/Angola?fullText=true
https://restcountries.eu/data/ago.svg
Processing Record 0 - (Antigua and Barbuda)
https://restcountries.eu/rest/v2/name/Antigua and Barbuda?fullText=true

https://restcountries.eu/data/gnq.svg
Processing Record 0 - (Eritrea)
https://restcountries.eu/rest/v2/name/Eritrea?fullText=true
https://restcountries.eu/data/eri.svg
Processing Record 0 - (Estonia)
https://restcountries.eu/rest/v2/name/Estonia?fullText=true
https://restcountries.eu/data/est.svg
Processing Record 0 - (Ethiopia)
https://restcountries.eu/rest/v2/name/Ethiopia?fullText=true
https://restcountries.eu/data/eth.svg
Processing Record 0 - (Fiji)
https://restcountries.eu/rest/v2/name/Fiji?fullText=true
https://restcountries.eu/data/fji.svg
Processing Record 0 - (Finland)
https://restcountries.eu/rest/v2/name/Finland?fullText=true
https://restcountries.eu/data/fin.svg
Processing Record 0 - (France)
https://restcountries.eu/rest/v2/name/France?fullText=true
https://restcountries.eu/data/fra.svg
Processing Record 0 - (Gabon)
https://restcountries.eu/rest/v2/name/Gabon?fullText=true
https://restcountries.eu/data/gab.svg
Processing Record 0 - (Gambia)
https://restcountries.eu/rest/v

Processing Record 0 - (Nauru)
https://restcountries.eu/rest/v2/name/Nauru?fullText=true
https://restcountries.eu/data/nru.svg
Processing Record 0 - (Nepal)
https://restcountries.eu/rest/v2/name/Nepal?fullText=true
https://restcountries.eu/data/npl.svg
Processing Record 0 - (Netherlands)
https://restcountries.eu/rest/v2/name/Netherlands?fullText=true
https://restcountries.eu/data/nld.svg
Processing Record 0 - (New Caledonia)
https://restcountries.eu/rest/v2/name/New Caledonia?fullText=true
https://restcountries.eu/data/ncl.svg
Processing Record 0 - (New Zealand)
https://restcountries.eu/rest/v2/name/New Zealand?fullText=true
https://restcountries.eu/data/nzl.svg
Processing Record 0 - (Nicaragua)
https://restcountries.eu/rest/v2/name/Nicaragua?fullText=true
https://restcountries.eu/data/nic.svg
Processing Record 0 - (Niger)
https://restcountries.eu/rest/v2/name/Niger?fullText=true
https://restcountries.eu/data/ner.svg
Processing Record 0 - (Nigeria)
https://restcountries.eu/rest/v2/name/

https://restcountries.eu/data/tuv.svg
Processing Record 0 - (Uganda)
https://restcountries.eu/rest/v2/name/Uganda?fullText=true
https://restcountries.eu/data/uga.svg
Processing Record 0 - (Ukraine)
https://restcountries.eu/rest/v2/name/Ukraine?fullText=true
https://restcountries.eu/data/ukr.svg
Processing Record 0 - (United Arab Emirates)
https://restcountries.eu/rest/v2/name/United Arab Emirates?fullText=true
https://restcountries.eu/data/are.svg
Processing Record 0 - (United Kingdom of Great Britain and Northern Ireland)
https://restcountries.eu/rest/v2/name/United Kingdom of Great Britain and Northern Ireland?fullText=true
https://restcountries.eu/data/gbr.svg
Processing Record 0 - (United Republic of Tanzania)
https://restcountries.eu/rest/v2/name/United Republic of Tanzania?fullText=true
https://restcountries.eu/data/tza.svg
Processing Record 0 - (United States of America)
https://restcountries.eu/rest/v2/name/United States of America?fullText=true
https://restcountries.eu/data/us

In [6]:
# Check the result
df.head()

,areaname,regname,country,devname,1980,1981,1982,1983,1984,1985,...,2010,2011,2012,2013,lat,lon,flag,capital,population,countrycode
0,Asia,Southern Asia,Afghanistan,Developing regions,16.0,39.0,39.0,47.0,71.0,340.0,...,1758.0,2203.0,2635.0,2004.0,33.000000,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF
1,Europe,Southern Europe,Albania,Developed regions,1.0,0.0,0.0,0.0,0.0,0.0,...,561.0,539.0,620.0,603.0,41.000000,20.0,https://restcountries.eu/data/alb.svg,Tirana,2886026,AL
2,Africa,Northern Africa,Algeria,Developing regions,80.0,67.0,71.0,69.0,63.0,44.0,...,4752.0,4325.0,3774.0,4331.0,28.000000,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ
3,Oceania,Polynesia,American Samoa,Developing regions,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-14.333333,-170.0,https://restcountries.eu/data/asm.svg,Pago Pago,57100,AS
4,Europe,Southern Europe,Andorra,Developed regions,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,42.500000,1.5,https://restcountries.eu/data/and.svg,Andorra la Vella,78014,AD



### Transpose Years from columns to rows

In [7]:
# Define a new DataFrame
new_df = pd.DataFrame(columns=['areaname', 'regname', 'country', 'devname','lat','lon','flag', \
                               'capital','population','countrycode','year','value'])

# Looping to get rows from the old df
for index, row in df.iterrows():
    # Looping to transpose years columns to rows
    for year in range(1980, 2014):
        newRow =  {'areaname':row['areaname'], 'regname':row['regname'], 'country': row['country'], \
                   'devname': row['devname'], 'lat': row['lat'],'lon': row['lon'], 'flag': row['flag'], \
                   'capital': row['capital'],'population':row['population'],'countrycode':row['countrycode'],\
                   'year': year, 'value' : round(row[str(year)])}
        print(newRow)
        # Add new line to the new DataFrame.
        new_df = new_df.append(newRow, ignore_index=True)

new_df.head()

{'areaname': 'Asia', 'regname': 'Southern Asia', 'country': 'Afghanistan', 'devname': 'Developing regions', 'lat': 33.0, 'lon': 65.0, 'flag': 'https://restcountries.eu/data/afg.svg', 'capital': 'Kabul', 'population': 27657145, 'countrycode': 'AF', 'year': 1980, 'value': 16}
{'areaname': 'Asia', 'regname': 'Southern Asia', 'country': 'Afghanistan', 'devname': 'Developing regions', 'lat': 33.0, 'lon': 65.0, 'flag': 'https://restcountries.eu/data/afg.svg', 'capital': 'Kabul', 'population': 27657145, 'countrycode': 'AF', 'year': 1981, 'value': 39}
{'areaname': 'Asia', 'regname': 'Southern Asia', 'country': 'Afghanistan', 'devname': 'Developing regions', 'lat': 33.0, 'lon': 65.0, 'flag': 'https://restcountries.eu/data/afg.svg', 'capital': 'Kabul', 'population': 27657145, 'countrycode': 'AF', 'year': 1982, 'value': 39}
{'areaname': 'Asia', 'regname': 'Southern Asia', 'country': 'Afghanistan', 'devname': 'Developing regions', 'lat': 33.0, 'lon': 65.0, 'flag': 'https://restcountries.eu/data/af

{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Albania', 'devname': 'Developed regions', 'lat': 41.0, 'lon': 20.0, 'flag': 'https://restcountries.eu/data/alb.svg', 'capital': 'Tirana', 'population': 2886026, 'countrycode': 'AL', 'year': 1984, 'value': 0}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Albania', 'devname': 'Developed regions', 'lat': 41.0, 'lon': 20.0, 'flag': 'https://restcountries.eu/data/alb.svg', 'capital': 'Tirana', 'population': 2886026, 'countrycode': 'AL', 'year': 1985, 'value': 0}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Albania', 'devname': 'Developed regions', 'lat': 41.0, 'lon': 20.0, 'flag': 'https://restcountries.eu/data/alb.svg', 'capital': 'Tirana', 'population': 2886026, 'countrycode': 'AL', 'year': 1986, 'value': 1}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Albania', 'devname': 'Developed regions', 'lat': 41.0, 'lon': 20.0, 'flag': 'https://restcountries.eu/data/alb.svg',

{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Algeria', 'devname': 'Developing regions', 'lat': 28.0, 'lon': 3.0, 'flag': 'https://restcountries.eu/data/dza.svg', 'capital': 'Algiers', 'population': 40400000, 'countrycode': 'DZ', 'year': 1985, 'value': 44}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Algeria', 'devname': 'Developing regions', 'lat': 28.0, 'lon': 3.0, 'flag': 'https://restcountries.eu/data/dza.svg', 'capital': 'Algiers', 'population': 40400000, 'countrycode': 'DZ', 'year': 1986, 'value': 69}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Algeria', 'devname': 'Developing regions', 'lat': 28.0, 'lon': 3.0, 'flag': 'https://restcountries.eu/data/dza.svg', 'capital': 'Algiers', 'population': 40400000, 'countrycode': 'DZ', 'year': 1987, 'value': 132}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Algeria', 'devname': 'Developing regions', 'lat': 28.0, 'lon': 3.0, 'flag': 'https://restcountries.eu/data

{'areaname': 'Oceania', 'regname': 'Polynesia', 'country': 'American Samoa', 'devname': 'Developing regions', 'lat': -14.33333333, 'lon': -170.0, 'flag': 'https://restcountries.eu/data/asm.svg', 'capital': 'Pago Pago', 'population': 57100, 'countrycode': 'AS', 'year': 1982, 'value': 0}
{'areaname': 'Oceania', 'regname': 'Polynesia', 'country': 'American Samoa', 'devname': 'Developing regions', 'lat': -14.33333333, 'lon': -170.0, 'flag': 'https://restcountries.eu/data/asm.svg', 'capital': 'Pago Pago', 'population': 57100, 'countrycode': 'AS', 'year': 1983, 'value': 0}
{'areaname': 'Oceania', 'regname': 'Polynesia', 'country': 'American Samoa', 'devname': 'Developing regions', 'lat': -14.33333333, 'lon': -170.0, 'flag': 'https://restcountries.eu/data/asm.svg', 'capital': 'Pago Pago', 'population': 57100, 'countrycode': 'AS', 'year': 1984, 'value': 0}
{'areaname': 'Oceania', 'regname': 'Polynesia', 'country': 'American Samoa', 'devname': 'Developing regions', 'lat': -14.33333333, 'lon': -

{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Andorra', 'devname': 'Developed regions', 'lat': 42.5, 'lon': 1.5, 'flag': 'https://restcountries.eu/data/and.svg', 'capital': 'Andorra la Vella', 'population': 78014, 'countrycode': 'AD', 'year': 1985, 'value': 0}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Andorra', 'devname': 'Developed regions', 'lat': 42.5, 'lon': 1.5, 'flag': 'https://restcountries.eu/data/and.svg', 'capital': 'Andorra la Vella', 'population': 78014, 'countrycode': 'AD', 'year': 1986, 'value': 2}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Andorra', 'devname': 'Developed regions', 'lat': 42.5, 'lon': 1.5, 'flag': 'https://restcountries.eu/data/and.svg', 'capital': 'Andorra la Vella', 'population': 78014, 'countrycode': 'AD', 'year': 1987, 'value': 0}
{'areaname': 'Europe', 'regname': 'Southern Europe', 'country': 'Andorra', 'devname': 'Developed regions', 'lat': 42.5, 'lon': 1.5, 'flag': 'https://restcountri

...

{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Western Sahara', 'devname': 'Developing regions', 'lat': 24.5, 'lon': -13.0, 'flag': 'https://restcountries.eu/data/esh.svg', 'capital': 'El Aaiún', 'population': 510713, 'countrycode': 'EH', 'year': 2008, 'value': 0}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Western Sahara', 'devname': 'Developing regions', 'lat': 24.5, 'lon': -13.0, 'flag': 'https://restcountries.eu/data/esh.svg', 'capital': 'El Aaiún', 'population': 510713, 'countrycode': 'EH', 'year': 2009, 'value': 0}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Western Sahara', 'devname': 'Developing regions', 'lat': 24.5, 'lon': -13.0, 'flag': 'https://restcountries.eu/data/esh.svg', 'capital': 'El Aaiún', 'population': 510713, 'countrycode': 'EH', 'year': 2010, 'value': 0}
{'areaname': 'Africa', 'regname': 'Northern Africa', 'country': 'Western Sahara', 'devname': 'Developing regions', 'lat': 24.5, 'lon': -13.0, 'flag': '

{'areaname': 'Asia', 'regname': 'Western Asia', 'country': 'Yemen', 'devname': 'Developing regions', 'lat': 15.0, 'lon': 48.0, 'flag': 'https://restcountries.eu/data/yem.svg', 'capital': "Sana'a", 'population': 27478000, 'countrycode': 'YE', 'year': 2007, 'value': 122}
{'areaname': 'Asia', 'regname': 'Western Asia', 'country': 'Yemen', 'devname': 'Developing regions', 'lat': 15.0, 'lon': 48.0, 'flag': 'https://restcountries.eu/data/yem.svg', 'capital': "Sana'a", 'population': 27478000, 'countrycode': 'YE', 'year': 2008, 'value': 133}
{'areaname': 'Asia', 'regname': 'Western Asia', 'country': 'Yemen', 'devname': 'Developing regions', 'lat': 15.0, 'lon': 48.0, 'flag': 'https://restcountries.eu/data/yem.svg', 'capital': "Sana'a", 'population': 27478000, 'countrycode': 'YE', 'year': 2009, 'value': 128}
{'areaname': 'Asia', 'regname': 'Western Asia', 'country': 'Yemen', 'devname': 'Developing regions', 'lat': 15.0, 'lon': 48.0, 'flag': 'https://restcountries.eu/data/yem.svg', 'capital': "Sa

{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zambia', 'devname': 'Developing regions', 'lat': -15.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zmb.svg', 'capital': 'Lusaka', 'population': 15933883, 'countrycode': 'ZM', 'year': 2008, 'value': 64}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zambia', 'devname': 'Developing regions', 'lat': -15.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zmb.svg', 'capital': 'Lusaka', 'population': 15933883, 'countrycode': 'ZM', 'year': 2009, 'value': 60}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zambia', 'devname': 'Developing regions', 'lat': -15.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zmb.svg', 'capital': 'Lusaka', 'population': 15933883, 'countrycode': 'ZM', 'year': 2010, 'value': 102}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zambia', 'devname': 'Developing regions', 'lat': -15.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zm

{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zimbabwe', 'devname': 'Developing regions', 'lat': -20.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zwe.svg', 'capital': 'Harare', 'population': 14240168, 'countrycode': 'ZW', 'year': 2008, 'value': 611}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zimbabwe', 'devname': 'Developing regions', 'lat': -20.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zwe.svg', 'capital': 'Harare', 'population': 14240168, 'countrycode': 'ZW', 'year': 2009, 'value': 508}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zimbabwe', 'devname': 'Developing regions', 'lat': -20.0, 'lon': 30.0, 'flag': 'https://restcountries.eu/data/zwe.svg', 'capital': 'Harare', 'population': 14240168, 'countrycode': 'ZW', 'year': 2010, 'value': 494}
{'areaname': 'Africa', 'regname': 'Eastern Africa', 'country': 'Zimbabwe', 'devname': 'Developing regions', 'lat': -20.0, 'lon': 30.0, 'flag': 'https://restcountries.

,areaname,regname,country,devname,lat,lon,flag,capital,population,countrycode,year,value
0,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1980,16
1,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1981,39
2,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1982,39
3,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1983,47
4,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1984,71


In [8]:
new_df.head(50)

,areaname,regname,country,devname,lat,lon,flag,capital,population,countrycode,year,value
0,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1980,16
1,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1981,39
2,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1982,39
3,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1983,47
4,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1984,71
5,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1985,340
6,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1986,496
7,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1987,741
8,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1988,828
9,Asia,Southern Asia,Afghanistan,Developing regions,33.0,65.0,https://restcountries.eu/data/afg.svg,Kabul,27657145,AF,1989,1076


In [11]:
# Save a new csv file
file = os.path.join("static", "data", "dataset.csv")
new_df.to_csv(file, encoding="utf-8", index=False) 

### Creating dataset to dropdown Global Areas

In [4]:
# Creating dataset to dropdown Global Areas option
file = os.path.join("static", "data", "dataset.csv")
df = pd.read_csv(file) #, encoding="latin1") 

aggregation = {"value": "sum"}
region = df.groupby(["areaname","devname","year"],axis=0).agg(aggregation)
print(region)


                                  value
areaname devname            year       
Africa   Developing regions 1980   3936
                            1981   4347
                            1982   3802
                            1983   2654
                            1984   2611
...                                 ...
Oceania  Developing regions 2009    322
                            2010    411
                            2011    322
                            2012    280
                            2013    239

[272 rows x 1 columns]


In [5]:
region.head()

value
areaname devname            year       
Africa   Developing regions 1980   3936
                            1981   4347
                            1982   3802
                            1983   2654
                            1984   2611

In [7]:
# Save the Global Areas csv file with devname
file = os.path.join("static", "data", "region.csv")
region.to_csv(file) 

In [8]:
# Select the Global Areas csv file with devname summarized (All)
file = os.path.join("static", "data", "dataset.csv")
df = pd.read_csv(file) #, encoding="latin1") 

aggregation = {"value": "sum"}
region = df.groupby(["areaname","year"],axis=0).agg(aggregation)
print(region)
# Save a new csv file
file = os.path.join("static", "data", "region2.csv")
region.to_csv(file) 


               value
areaname year       
Africa   1980   3936
         1981   4347
         1982   3802
         1983   2654
         1984   2611
...              ...
Oceania  2009   1860
         2010   1834
         2011   1548
         2012   1679
         2013   1775

[204 rows x 1 columns]


### Creating data for top 5 countries per region

In [2]:
# Select the top 5 summarizing with devname column (developing and developed areas).
countries_filter = [ 'Egypt' ,'Morocco' ,'Algeria' ,'Somalia' ,'South Africa' ,'India' ,'China','Philippines'  \
                    ,'Pakistan' ,'Iran (Islamic Republic of)' ,'United Kingdom of Great Britain and Northern Ireland' \
                    ,'Poland' ,'France' ,'Romania' ,'Portugal' ,'Jamaica' ,'Haiti' ,'Guyana' ,'Colombia' ,'Mexico' \
                    ,'Australia' ,'Fiji' ,'New Zealand' ,'Tonga' ,'Samoa']


file = os.path.join("static", "data", "dataset.csv")
df = pd.read_csv(file) 


top_5_countries = df[df.country.isin(countries_filter)]
top_5_countries.head()


,areaname,regname,country,devname,lat,lon,flag,capital,population,countrycode,year,value
68,Africa,Northern Africa,Algeria,Developing regions,28.0,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ,1980,80
69,Africa,Northern Africa,Algeria,Developing regions,28.0,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ,1981,67
70,Africa,Northern Africa,Algeria,Developing regions,28.0,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ,1982,71
71,Africa,Northern Africa,Algeria,Developing regions,28.0,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ,1983,69
72,Africa,Northern Africa,Algeria,Developing regions,28.0,3.0,https://restcountries.eu/data/dza.svg,Algiers,40400000,DZ,1984,63


In [4]:
file = os.path.join("static", "data", "top_5_countries_devname.csv")
top_5_countries.to_csv(file, index=False) 

In [5]:
# Select the top 5 summarizing without devname column.
aggregation = {"value": "sum"}
top_5_countries_sum = top_5_countries.groupby(['areaname' ,'regname' ,'country','lat' ,'lon' ,'flag' ,'capital' ,'population' ,'countrycode' ,'year'],axis=0).agg(aggregation)

# Save a new csv file
file = os.path.join("static", "data", "top_5_countries_all.csv")
top_5_countries_sum.to_csv(file) 
top_5_countries_sum.head()

value
areaname regname        country lat  lon  flag                                  capital   population countrycode year       
Africa   Eastern Africa Somalia 10.0 49.0 https://restcountries.eu/data/som.svg Mogadishu 11079000   SO          1980      7
                                                                                                                 1981     10
                                                                                                                 1982      7
                                                                                                                 1983     12
                                                                                                                 1984     14

### Creating data for top 10 countries in the World (Developed and Developing areas)


In [9]:
# Select the top 10 with devname column (developing and developed areas).
countries_filter = ['United Kingdom of Great Britain and Northern Ireland' ,'United States of America' ,'Poland' , \
                    'France' ,'Romania' ,'Portugal' ,'Germany' ,'Russian Federation' ,'Ukraine' ,'Japan' ,'India' ,\
                    'China' ,'Philippines' ,'Pakistan' ,'Iran (Islamic Republic of)' ,'Sri Lanka' ,'Republic of Korea' ,\
                    'Lebanon' ,'Jamaica' ,'Viet Nam']

file = os.path.join("static", "data", "dataset.csv")
df = pd.read_csv(file)  


top_10_countries = df[df.country.isin(countries_filter)]
top_10_countries.head()



,areaname,regname,country,devname,lat,lon,flag,capital,population,countrycode,year,value
1190,Asia,Eastern Asia,China,Developing regions,35.0,105.0,https://restcountries.eu/data/chn.svg,Beijing,1377422166,CN,1980,5123
1191,Asia,Eastern Asia,China,Developing regions,35.0,105.0,https://restcountries.eu/data/chn.svg,Beijing,1377422166,CN,1981,6682
1192,Asia,Eastern Asia,China,Developing regions,35.0,105.0,https://restcountries.eu/data/chn.svg,Beijing,1377422166,CN,1982,3308
1193,Asia,Eastern Asia,China,Developing regions,35.0,105.0,https://restcountries.eu/data/chn.svg,Beijing,1377422166,CN,1983,1863
1194,Asia,Eastern Asia,China,Developing regions,35.0,105.0,https://restcountries.eu/data/chn.svg,Beijing,1377422166,CN,1984,1527


In [10]:
file = os.path.join("static", "data", "top_10_countries_devname.csv")
top_10_countries.to_csv(file, index=False) 

In [12]:
# Select the top 10 summarizing without devname column.
countries_filter = ['India' ,'China' ,'United Kingdom of Great Britain and Northern Ireland' ,'Philippines' , \
                    'Pakistan' ,'United States of America' ,'Iran (Islamic Republic of)' ,'Sri Lanka' ,'Republic of Korea' ,\
                    'Poland']

top_10_countries = df[df.country.isin(countries_filter)]


aggregation = {"value": "sum"}
top_10_countries_sum = top_10_countries.groupby(['areaname' ,'regname' ,'country','lat' ,'lon' ,'flag' ,'capital' ,'population' ,'countrycode' ,'year'],axis=0).agg(aggregation)

# Save a new csv file
file = os.path.join("static", "data", "top_10_countries_all.csv")
top_10_countries_sum.to_csv(file) 
top_10_countries_sum.head()

value
areaname regname      country lat  lon   flag                                  capital population countrycode year       
Asia     Eastern Asia China   35.0 105.0 https://restcountries.eu/data/chn.svg Beijing 1377422166 CN          1980   5123
                                                                                                              1981   6682
                                                                                                              1982   3308
                                                                                                              1983   1863
                                                                                                              1984   1527